# **Unveiling Gender Bias: Statistical Analyses of Salary Prediction Data**
### *By Dr. Adnan Masood*

## **Purpose of this Notebook**
In this notebook, we will explore salary data and see if there is any difference in salaries for people based on their gender. We will do several simple analyses (ELI5 – "Explain Like I'm 5") to see if there is any unfairness in how salaries are distributed.

We are using ELI5 style explanations so that complex ideas become easier to digest:
- We will break down statistical methods, such as t-tests, into simpler terms.
- We will showcase various data visualizations to spot any inequality or bias.
- We will also look into how to potentially fix or reduce bias (called **debiasing**), with ample detail provided so each step is clear.


## **Why This Matters**
Bias in data can lead to unfair outcomes, especially if a model or decision-making process is built on biased information. By investigating data:
- We can **identify** if one group is at a disadvantage.
- We can **measure** how large that disadvantage might be.
- We can **rectify** or reduce that disadvantage using **debiasing** techniques.

When salaries are in question, addressing bias ensures fair pay practices and helps foster workplace equality.

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mohithsairamreddy_salary_data_path = kagglehub.dataset_download('mohithsairamreddy/salary-data')
print('Data source import complete.')

## **Importing Libraries**
We will use the following Python libraries:
- **pandas** for data manipulation (tables, CSV reading, etc.)
- **numpy** for numerical operations and array handling
- **matplotlib** and **seaborn** for data visualization (plots, histograms, etc.)
- **scipy.stats** for statistical tests such as t-tests and Theil index calculations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import entropy

## **1. Dataset Description**
We will load the dataset and do a quick examination of its structure. Specifically:
1. **Peek at the first few rows** to understand how the data is organized.
2. **Check summary statistics** (like mean, min, max) to detect any anomalies.
3. **Verify data types** to confirm columns that we think are numeric are indeed numeric.
4. **Look for missing values** because they can cause problems in analysis or model training.

In [ ]:
# TODO: Load the dataset into salary_df.
# This step is critical for reading our CSV file into a pandas DataFrame.
# We'll then inspect the data to ensure it's loaded properly.
# HINT: Use pd.read_csv("/kaggle/input/salary-data/Salary_Data.csv") if your data is stored there.

##################################
# Replace the None with your code.
##################################
salary_df = None  # TODO

salary_df

### **Summary Statistics**
Summary statistics show us how each column is distributed. For example, we can see **mean** salary, **minimum** and **maximum** salary, and also **standard deviation**, which tells us how spread out the salaries are.

In [ ]:
# TODO: Use salary_df.describe() to see the data summary.
# HINT: dataset_summary = salary_df.describe()

##################
# Fill in below:
##################
dataset_summary = None  # TODO
print("Summary Statistics of the Salary Prediction Dataset:")
dataset_summary

### **Check Data Types**
We want to confirm which columns are numeric (like `Salary`) and which are categorical (like `Gender`). This helps ensure later steps (like building models) won't fail due to incorrect data types.

In [ ]:
# TODO: Check the dtypes of columns using .dtypes.
# For example: salary_df.dtypes

##################
# Fill in below:
##################
salary_df.dtypes  # TODO

### **Check for Missing Values**
Missing data can arise if an employee's salary or years of experience wasn't recorded. We can address these by dropping rows or filling them if we have a valid strategy.

In [ ]:
# TODO: Check for missing values.
# We can do: salary_df.isnull().sum() to see how many missing values exist in each column.

##################
# Fill in below:
##################
null_values = None  # TODO
print(null_values)

In [ ]:
# TODO: If needed, drop or fix those missing values.
# You might use salary_df.dropna() to just remove them, or fill them in with some average.

##################
# Fill in below:
##################
salary_df = None  # TODO dropna or handle missing as appropriate

# Double-check null values again:
null_values = salary_df.isnull().sum()
print(null_values)

## **2. Exploratory Data Analysis (EDA)**
In this phase, we visualize the data using charts. The idea is to see the distribution of each variable and spot any big differences between groups. By simply looking at these visual plots, we can gain immediate intuition about potential biases.

In [ ]:
# Histograms
# ELI5: A histogram counts how many data points fall into each interval ('bucket').
# We can see if the data is skewed or if there's a wide/narrow range.

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 15))

sns.histplot(salary_df['Age'], bins=20, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('Distribution of Age')

sns.histplot(salary_df['Years of Experience'], bins=20, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('Distribution of Years of Experience')

sns.histplot(salary_df['Salary'], bins=20, kde=True, ax=axes[2], color='green')
axes[2].set_title('Distribution of Salary')

plt.tight_layout()
plt.show()

In [ ]:
# Box plots
# ELI5: A box plot helps us see quartiles and outliers.
# The box shows where the bulk of the data is, and the 'whiskers' can show extremes.

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 15))

sns.boxplot(x=salary_df['Age'], ax=axes[0], color='skyblue')
axes[0].set_title('Box Plot of Age')

sns.boxplot(x=salary_df['Years of Experience'], ax=axes[1], color='salmon')
axes[1].set_title('Box Plot of Years of Experience')

sns.boxplot(x=salary_df['Salary'], ax=axes[2], color='green')
axes[2].set_title('Box Plot of Salary')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot for Age vs. Salary
# Here, each dot is an employee.
# We color them by Gender so we can see if one group tends to cluster in certain areas.

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Salary', data=salary_df, hue='Gender', palette='viridis', alpha=0.7)
plt.title('Scatter Plot of Age vs. Salary')
plt.show()

### **Gender-Related Visualizations**
Next, let's see how many males vs. females are in the dataset, and also how their years of experience, age, and salaries compare. This helps us detect if one group might be underrepresented or might systematically earn more.

In [ ]:
gender_counts = salary_df['Gender'].value_counts()
plt.bar(gender_counts.index, gender_counts)
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Distribution of Gender')
plt.show()

In [ ]:
# Check the distribution of Years of Experience grouped by Gender.
# This can show if one group typically has more or less experience.

plt.figure(figsize=(10, 6))
sns.boxplot(x='Gender', y='Years of Experience', data=salary_df, palette='Set2')
plt.title('Experience Distribution by Gender')
plt.show()

In [ ]:
# Check the distribution of Age grouped by Gender.
# If there's a big age difference between groups, it might also affect salaries.

plt.figure(figsize=(10, 6))
sns.boxplot(x='Gender', y='Age', data=salary_df, palette='Set2')
plt.title('Age Distribution by Gender')
plt.show()

In [ ]:
# Check how Salary is distributed by Gender.
# A big shift in the box/whiskers might indicate a pay gap.

plt.figure(figsize=(10, 6))
sns.boxplot(x='Gender', y='Salary', data=salary_df, palette='Set2')
plt.title('Salary Distribution by Gender')
plt.show()

## **3. Descriptive Statistics by Gender**
Seeing basic numeric descriptions (like count, mean, median, etc.) for each gender can tell us if the average salary is drastically different. We also look at how consistent these salaries are (using standard deviation).

In [ ]:
gender_groups = salary_df.groupby('Gender')['Age']
summary_statistics = gender_groups.agg(['count', 'mean', 'median', 'std'])
print("Summary Statistics for Age by Gender:")
summary_statistics

### **Salary Stats by Gender**
This is especially pertinent since we are looking for a potential salary gap.

In [ ]:
gender_groups = salary_df.groupby('Gender')['Salary']
summary_statistics = gender_groups.agg(['count', 'mean', 'median', 'std'])
print("Summary Statistics for Salary by Gender:")
summary_statistics

## **4. Hypothesis Testing**
### **T-Test**
We use a two-sample t-test to see if the average salary for males is significantly different from the average salary for females. In simpler terms:
- **Null Hypothesis (H0):** "There is no difference in salary based on gender."
- **Alternative Hypothesis (H1):** "There is a difference in salary based on gender."

If the p-value is below a typical threshold (like 0.05), it suggests the difference is not just due to random chance.

In [ ]:
male_salaries = salary_df[salary_df['Gender'] == 'Male']['Salary']
female_salaries = salary_df[salary_df['Gender'] == 'Female']['Salary']

t_stat, p_value = ttest_ind(male_salaries, female_salaries)
print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. Salaries differ significantly by gender.")
else:
    print("Fail to reject the null hypothesis. No significant difference found.")

### **Cohen's d**
While a t-test determines if the difference is statistically significant, **Cohen's d** tells us how large or meaningful that difference is:
- ~0.2 = small effect
- ~0.5 = medium effect
- ~0.8 (or above) = large effect


In [ ]:
male_salaries = salary_df[salary_df['Gender'] == 'Male']['Salary']
female_salaries = salary_df[salary_df['Gender'] == 'Female']['Salary']

t_stat, p_value = ttest_ind(male_salaries, female_salaries)
mean_difference = male_salaries.mean() - female_salaries.mean()
pooled_std_dev = (
    ((len(male_salaries) - 1) * male_salaries.var()) + ((len(female_salaries) - 1) * female_salaries.var())
) / (len(male_salaries) + len(female_salaries) - 2)
cohen_d = mean_difference / (pooled_std_dev ** 0.5)

print(f"Cohen's d: {cohen_d}")

## **5. Theil Index**
The **Theil Index** is an entropy measure that shows how unequal or uneven a distribution is. If the Theil Index is very high for one group, that group’s salaries might have a wider spread of values (some extremely high, some extremely low).

In [ ]:
male_salaries = salary_df[salary_df['Gender'] == 'Male']['Salary']
female_salaries = salary_df[salary_df['Gender'] == 'Female']['Salary']

theil_male = entropy(male_salaries)
theil_female = entropy(female_salaries)
print(f"Theil Index for Male: {theil_male}")
print(f"Theil Index for Female: {theil_female}")

## **6. Demographic Parity**
Demographic parity checks if each group has about the same rate of a “positive outcome.” Here, we define crossing a certain salary threshold (e.g., $100k) as that outcome. If one group is significantly less likely to cross that threshold, that might indicate a biased scenario.

In [ ]:
salary_threshold = 100000
salary_df.loc[:, 'Outcome'] = (salary_df['Salary'] > salary_threshold).astype(int)

male_data = salary_df[salary_df['Gender'] == 'Male']
female_data = salary_df[salary_df['Gender'] == 'Female']

parity_male = male_data['Outcome'].mean()
parity_female = female_data['Outcome'].mean()

print(f"Demographic Parity for Male: {parity_male}")
print(f"Demographic Parity for Female: {parity_female}")

## **7. Disparate Impact**
Disparate impact looks at the ratio of positive outcomes for a group relative to the overall positive outcome rate. If the ratio is close to 1, it suggests fairness. A ratio much below (or above) 1 signals potential bias.

In [ ]:
male_data = salary_df[salary_df['Gender'] == 'Male']
female_data = salary_df[salary_df['Gender'] == 'Female']

disparate_impact_male = male_data['Outcome'].mean() / salary_df['Outcome'].mean()
disparate_impact_female = female_data['Outcome'].mean() / salary_df['Outcome'].mean()

print(f"Disparate Impact for Male: {disparate_impact_male}")
print(f"Disparate Impact for Female: {disparate_impact_female}")

# **Exercise 1: Train a Simple Model to Predict Salary**
In this exercise, we will create a straightforward regression model that tries to predict an employee’s salary based on their attributes. Once the model is trained, we will pass in the same values for age and years of experience, but only change the **Gender** from female to male to see if the model gives different salary predictions.

**Goal:**
1. Split the data into features (Age, Experience, Gender, etc.) and the target (Salary).
2. Fit a regression model (like Linear Regression).
3. Predict the salary for a hypothetical person with certain parameters:
   - Age = 30
   - Years of Experience = 5
   - Gender = Female
4. Predict the salary for the **same** parameters except Gender = Male.
5. Compare the results to see how big the difference is.

### **Hints**
- You might use `train_test_split` from `sklearn.model_selection` to break data into training and testing sets.
- You might use `LinearRegression` from `sklearn.linear_model`.
- Be sure to handle the `Gender` column properly. Convert it to numeric if needed (e.g., 0 for Female, 1 for Male).

Examining these differences helps us identify if the model is learning a gender-based bias.

In [ ]:
# TODO: Implement your model training
# Steps:
# 1. Select relevant features from salary_df (Age, Years of Experience, Gender?).
# 2. Convert Gender to numeric if necessary (like using pd.get_dummies).
# 3. Split into train/test.
# 4. Fit a LinearRegression.
# 5. Make predictions for two hypothetical individuals (same Age, same Years of Exp, different Gender).
# 6. Print out both predicted salaries.

##############################################
# EXERCISE: Fill in your code below
##############################################

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# 1. Prepare X, y
# 2. train_test_split
# 3. model.fit
# 4. hypothetical_female = ...
# 5. hypothetical_male = ...
# 6. Compare results

# TODO: Complete below.


After you run these predictions, note the difference between male and female salaries for the exact same input attributes (other than gender). If there is a gap, it may reflect biases in the data or how the model interprets those data points.

# **Exercise 2: Debias the Dataset (Using a Reweighing Approach)**
In this exercise, rather than removing the `Gender` column, we will use a **reweighing** strategy to ensure each gender group receives a fairer representation during the training process. This method assigns **larger weights** to underrepresented or disadvantaged groups so the model pays more "attention" to them when learning.

## **What is Reweighing?**
- **Goal:** Adjust the importance of training examples so that the dataset no longer (or less) reflects historical imbalances.
- **Mechanism:**  
  1. Identify each record’s *group* (e.g., "Male" or "Female").  
  2. Compute how often that group occurs vs. how often it *should* occur under "perfect fairness" assumptions.  
  3. Apply a *weight* to each record inversely proportional to how over- or underrepresented it is.

By doing this, we do **not** remove any columns, and the model can still see the `Gender`. But it “pays more attention” to the minority group’s samples to reduce bias from historical imbalance.

## **Detailed Steps**
### **1. Separate the Dataset by Gender**
```python
import pandas as pd

# Suppose salary_df is your original DataFrame with columns like:
# ['Age', 'Years of Experience', 'Gender', 'Salary']

male_df = salary_df[salary_df['Gender'] == 'Male']
female_df = salary_df[salary_df['Gender'] == 'Female']
```
1. We split into subsets for each group. If there are more categories (e.g., Non-binary/Other), you’d extend this logic accordingly.

### **2. Calculate Group Proportions**
```python
# Count how many total examples in the dataset
total_count = len(salary_df)

# Count how many in each group
male_count = len(male_df)
female_count = len(female_df)

# Current (observed) proportions
male_prop = male_count / total_count
female_prop = female_count / total_count

print("Current male proportion:", male_prop)
print("Current female proportion:", female_prop)
```

### **3. Decide on "Desired" Proportions**
```python
desired_male_prop = 0.5
desired_female_prop = 0.5
```
Typically, we aim for a balanced representation (50-50) or whatever distribution we believe is fair.

### **4. Compute the Weights**
```python
weight_for_male   = desired_male_prop   / male_prop
weight_for_female = desired_female_prop / female_prop

print("Weight for male samples:", weight_for_male)
print("Weight for female samples:", weight_for_female)
```
These weights tell us how much to up- or down-weight each record. If a group is overrepresented, that group’s weight is < 1. If underrepresented, weight is > 1.

### **5. Assign Weights to Each Row**
```python
salary_df['sample_weight'] = salary_df['Gender'].apply(
    lambda g: weight_for_male if g == 'Male' else weight_for_female
)
```
Now every row has a numeric `sample_weight` that indicates how strongly it should influence the model’s training.

### **6. Train a Model with `sample_weight`**
```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Suppose we want to predict 'Salary' from ['Age', 'Years of Experience', 'Gender'].
# 1. Separate features (X) and target (y).
X = salary_df[['Age', 'Years of Experience', 'Gender']]  # or encode Gender in numeric form if needed
y = salary_df['Salary']

# 2. Train-test split
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y, salary_df['sample_weight'], test_size=0.2, random_state=42
)

# 3. Create and train the model
model = LinearRegression()
model.fit(X_train, y_train, sample_weight=w_train)  # <-- important step

# 4. Evaluate the model
score = model.score(X_test, y_test, sample_weight=w_test)
print("Model R^2 on test data (with reweighing):", score)
```
The model now sees that female samples might have higher weight if they were underrepresented. This can help reduce biases that would favor the majority group.

### **7. Compare Predictions for Male vs. Female**
You can again do a hypothetical input test (e.g., Age=30, Experience=5) and vary `Gender` from "Male" to "Female" to observe whether the difference in predicted salary is smaller now than it was **before** reweighing.

## **Key Takeaways**
- **Reweighing** is a *pre-processing* fairness strategy. By adjusting the importance of each record, the model becomes more balanced towards the desired distribution.
- **This approach** does not hide `Gender` from the model. Instead, it leverages the gender information to correct representation.
- While reweighing can reduce group-level bias, it does *not* guarantee fairness on every individual level. Other advanced methods or fairness libraries like **Fairlearn** or **AIF360** may be used for more comprehensive solutions.

By carefully **reweighing** the samples, we can help the model treat underrepresented groups more fairly, potentially reducing the pay gap it predicts for different genders.